In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from pandas import Series, DataFrame
from statsmodels.formula.api import ols
from statsmodels.api import MNLogit
import statsmodels.discrete.discrete_model as sm
from sklearn import preprocessing
import math

In [13]:
import random
random.seed(0)

In [14]:
pd.options.mode.chained_assignment = None

In [15]:
from sklearn.linear_model import LogisticRegression

# Data Cleaning

In [16]:
data = pd.read_csv('STARD/S1_data_kNN_2.csv', index_col='Patient')

In [17]:
def read_data():
    data = pd.read_csv('STARD/S1_data_kNN_2.csv', index_col='Patient')


    #data cleaning
    data['L3_treatment'] = data['L3_treatment'].fillna(0)  #define no-treatment in the second stage as treatment 0
    data['L3_End_QCTOT'] = data['L3_End_QCTOT'].fillna(data['L3_Int_QCTOT'])  #define the outcome for patients not treated in the second stage

    #drop the two rows where we treat someone with QCTOT == 5
    data.L3_treatment[1246] = None
    data.L3_treatment[3151] = None


    data = data.dropna()

    data.L2_Int_QCTOT = -data.L2_Int_QCTOT
    data.L2_End_QCTOT = -data.L2_End_QCTOT
    data.L3_Int_QCTOT = -data.L3_Int_QCTOT
    data.L3_End_QCTOT = -data.L3_End_QCTOT
    data['dropped'] = None
    data['p1'] = None
    data['p0'] = None
    data['p2'] = None
    data['p'] = None
    data['weight'] = None
    data['Y'] = data.L2_End_QCTOT + data.L3_End_QCTOT
    data['Y_impute'] = None

    data['A1_L2_Int_QCTOT'] = data.L2_treatment * data.L2_Int_QCTOT
    data['A1_L1_slope'] = data.L2_treatment * data.L1_slope
    data['A1_L2_preference'] = data.L2_treatment * data.L2_preference
    
    return data

## Compute Propensity

In [18]:
def compute_propensity_weight(data, tau=0):
    
    features = ['L2_Int_QCTOT','L1_slope','L2_preference']
    p1 = LogisticRegression().fit(data[features], data.L2_treatment == 1).predict_proba(data[features])[:,1]
    data.p1 = p1*(data.L2_treatment == 1) + (1-p1)*(data.L2_treatment == -1)
    
    #dropout/stay probability
    subdata = data[-data.L2_End_QCTOT > 5]
    features = ['L2_Int_QCTOT','L1_slope','L2_preference','L2_treatment', 'L2_End_QCTOT', 'L2_slope', 'A1_L2_Int_QCTOT','A1_L1_slope','A1_L2_preference']
    p0 = LogisticRegression(max_iter=1000).fit(subdata[features], 1-subdata['L3_Indicator'])
    p0 = p0.predict_proba(data[features])[:,1]
    data.p0 = p0*(data.L3_Indicator == 0) + (1-p0)*(data.L3_Indicator == 1)
    data.p0[-data.L2_End_QCTOT < 6] = 1
    
    # the prob of l3 = 1 given l2 = 1
    subdata = data[(data.L3_Indicator == 1) & (data.L2_treatment == 1)]
    features = ['L2_Int_QCTOT','L1_slope','L2_preference','L2_End_QCTOT', 'L2_slope', 'L3_preference']
    p2 = LogisticRegression().fit(subdata[features], subdata.L3_treatment == 1)
    p2 = p2.predict_proba(data[features])[:,1]
    data.p2 = p2 * (data.L3_treatment == 1) * (data.L2_treatment == 1) + (1-p2) * (data.L3_treatment == -1) * (data.L2_treatment == 1) 
    data.p2[data.p2==0] = 1
    
    data['p'] = data.p1 * data.p0 * data.p2
    
    #clip propensity with tau
    data.p = data.p.clip(tau,1)
    data.weight = 1/data.p
    
    return data

In [19]:
def upweighted_data(tau, random_state, n = 30000):
    data = read_data()
    data = compute_propensity_weight(data, tau)
    data = data.sample(n, replace=True, weights='weight', random_state=random_state)
    data.index = np.arange(len(data))
    return data

# Algorithms

## Helper Functions and Parameters

In [20]:
def compute_mean_reward(data):
    Reward = data.Y[data.dropped == False]     
    mean_reward = Reward.mean()
    return mean_reward/2

In [21]:
tau = 0
d = 3
n = 5000
n_reps = 10

## Logging

In [22]:
data = read_data()
data = compute_propensity_weight(data, tau = 0)
sum(((data.L3_treatment != 0) | (data.L3_Int_QCTOT > -6))*data.Y/data.p0)/sum(((data.L3_treatment != 0) | (data.L3_Int_QCTOT > -6))/data.p0) /2

-8.437883566138938

## DTR Bandit

In [30]:
def compute_forced_pulls_new(q, K, d):
    T = 20000
    forced_index_minus1minus1 = np.zeros(T, dtype = bool)
    forced_index_minus1minus1[0:d] = True
    forced_index_11 = np.zeros(T, dtype = bool)
    forced_index_11[d:(2*d)] = True
    forced_index_1minus1 = np.zeros(T, dtype = bool)
    
    for i in range(16):
        for j in range(q):
            if (2**i - 1) * K * q + j > 2*d - 1 and (2**i - 1) * K * q + j < T:
                forced_index_minus1minus1[(2**i - 1) * K * q + j] = True
            if (2**i - 1) * K * q + q + j> 2*d - 1 and (2**i - 1) * K * q + q + j < T:
                forced_index_1minus1[(2**i - 1) * K * q + q + j] = True
            if (2**i - 1) * K * q + 2*q + j> 2*d - 1 and (2**i - 1) * K * q + 2*q + j < T:
                forced_index_11[(2**i - 1) * K * q + 2*q + j] = True
    return forced_index_minus1minus1, forced_index_1minus1, forced_index_11

In [31]:
def compute_beta_tilde_linearQ(data):
    beta_hat_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', 
                           data= data[(data.dropped == False)  & (data.L3_treatment == -1) & (data.forced == True)]).fit() 
    beta_hat_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', 
                      data= data[(data.dropped == False)  & (data.L3_treatment == 1) & (data.forced == True)]).fit()
    
    # impute Y values for first stage
    data.Y_impute = data.L2_End_QCTOT + (-data.L2_End_QCTOT<=5)*data.L2_End_QCTOT +  ((-data.L2_End_QCTOT>5) & (data.L2_treatment == 1))* np.array([beta_hat_minus12.predict(data), beta_hat_12.predict(data)]).max(axis=0) + ((-data.L2_End_QCTOT>5) & (data.L2_treatment == -1))*beta_hat_minus12.predict(data)
    
    beta_hat_minus11 = ols('Y_impute ~ L2_Int_QCTOT + L1_slope + L2_preference', 
                           data= data[(data.dropped == False)  & (data.L2_treatment == -1) & (data.forced == True)]).fit() 
    beta_hat_11 = ols('Y_impute ~ L2_Int_QCTOT + L1_slope + L2_preference', 
                      data= data[(data.dropped == False)  & (data.L2_treatment == 1) & (data.forced == True)]).fit()     
    return beta_hat_minus11, beta_hat_11, beta_hat_minus12, beta_hat_12

In [32]:
def tilde_gap_big(beta_minus1, beta_1, X_i, delta):
    if beta_minus1.predict(X_i).iloc[0] - beta_1.predict(X_i).iloc[0] > delta:
        return True
    if beta_1.predict(X_i).iloc[0] - beta_minus1.predict(X_i).iloc[0] > delta:
        return True
    return False

In [33]:
def compute_beta_linearQ(data):
    beta_hat_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', 
                           data= data[(data.dropped == False)  & (data.L3_treatment == -1)]).fit() 
    beta_hat_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', 
                      data= data[(data.dropped == False)  & (data.L3_treatment == 1)]).fit()
    
    # impute Y values for first stage
    data.Y_impute = data.L2_End_QCTOT + (-data.L2_End_QCTOT<=5)*data.L2_End_QCTOT +  ((-data.L2_End_QCTOT>5) & (data.L2_treatment == 1))* np.array([beta_hat_minus12.predict(data), beta_hat_12.predict(data)]).max(axis=0) + ((-data.L2_End_QCTOT>5) & (data.L2_treatment == -1))*beta_hat_minus12.predict(data)
    
    beta_hat_minus11 = ols('Y_impute ~ L2_Int_QCTOT + L1_slope + L2_preference', 
                           data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
    beta_hat_11 = ols('Y_impute ~ L2_Int_QCTOT + L1_slope + L2_preference', 
                      data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit()     
    return beta_hat_minus11, beta_hat_11, beta_hat_minus12, beta_hat_12

In [34]:
def choose_optimal_action(beta_minus1, beta_1, X_i):
    if beta_minus1.predict(X_i).iloc[0] > beta_1.predict(X_i).iloc[0]:
        A = -1
    else:
        A = 1
    return A

In [35]:
def dtr_linear_Q(q: int, K: int, d: int, n: int, delta):                
    
    data['dropped'] = None
    data['forced'] = None
    n_total = 0 #total number of points
    forced_index_minus1minus1, forced_index_1minus1, forced_index_11 = compute_forced_pulls_new(q, K, d)

    for i in data.index:
    
        # compute our chosen arm
        if forced_index_11[n_total]:
            A_1 = 1
            A_2 = 1 
            data.forced[i] = True
        elif forced_index_minus1minus1[n_total]:
            A_1 = -1
            A_2 = -1
            data.forced[i] = True
        elif forced_index_1minus1[n_total]:
            A_1 = 1
            A_2 = -1
            data.forced[i] = True
        else:     
            data.forced[i] = False
            X_i = data[data.index == i]
            if tilde_gap_big(beta_tilde_minus11, beta_tilde_11, X_i, delta):
                A_1 = choose_optimal_action(beta_tilde_minus11, beta_tilde_11, X_i) 
            else:
                A_1 = choose_optimal_action(beta_hat_minus11, beta_hat_11, X_i) 
            if - data.L2_End_QCTOT[i] < 6:
                A_2 = 0
            elif A_1 == -1:
                A_2 = -1
            else:
                if tilde_gap_big(beta_tilde_minus12, beta_tilde_12, X_i, delta/2):  # should we try delta/2?
                    A_2 = choose_optimal_action(beta_tilde_minus12, beta_tilde_12, X_i)
                else:
                    A_2 = choose_optimal_action(beta_hat_minus12, beta_hat_12, X_i)
    
        # if disagrees, we drop the point; else, we update estimators
        if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
            data.dropped[i] = False
            n_total = n_total + 1         
            #estimate hat parameters
            if n_total >= 2*d:           
                beta_hat_minus11, beta_hat_11, beta_hat_minus12, beta_hat_12 = compute_beta_linearQ(data)  
                if data.forced[i]:
                    beta_tilde_minus11, beta_tilde_11, beta_tilde_minus12, beta_tilde_12 = compute_beta_tilde_linearQ(data)  
        else:
            data.dropped[i] = True 
        
        if n_total >= n:
            break
            
    # if we don't get enough samples, we omit this instance
    if n_total<n:
        print(n_total)
        return
                
    mean_reward = compute_mean_reward(data)
    return mean_reward

In [ ]:
K = 3
q_list = [1,2]
delta_list = [10, 20, 50]
dtr = {}
for i in range(n_reps):
    
    data = upweighted_data(tau=0, random_state = i)
    
    for q in q_list:
        for delta in delta_list:
            cur = dtr_linear_Q(q, K, d, n, delta)
            if (q, delta) in dtr:
                dtr[(q, delta)].append(cur)
            else:
                dtr[(q, delta)] = [cur]

In [30]:
for key in dtr:
    print(key, np.mean(dtr[key]))

(1, 10) -7.68006
(1, 20) -7.561490000000001
(1, 50) -7.552130000000001
(2, 10) -7.68764
(2, 20) -7.603649999999999
(2, 50) -7.547399999999999


## Epsilon Greedy

In [36]:
def epsilon_greedy(eps: int, d:int, n: int, random_state):
    data['dropped'] = None
    n_total = 0 #total number of points
    
    # generate random actions
    rng1 = np.random.RandomState(random_state*2)
    rng2 = np.random.RandomState(random_state*2+1)
    temp1 = rng1.binomial(1, eps, size = 30000)
    temp2 = rng2.binomial(1, eps, size = 30000)  

    for i in data.index:
    
        # compute our chosen arm
        if n_total < d:
            A_1 = -1
            A_2 = -1        
        elif n_total < 2*d:
            A_1 = 1
            A_2 = 1
        elif n_total < 3*d:
            A_1 = 1
            A_2 = -1
        else:    
            X_i = data[data.index == i]
            A_1 = choose_optimal_action(beta_hat_minus11, beta_hat_11, X_i)
            if temp1[i]:
                A_1 = -A_1
                
            if - data.L2_End_QCTOT[i] < 6:
                A_2 = 0
            elif A_1 == -1:
                A_2 = -1
            else:
                A_2 = choose_optimal_action(beta_hat_minus12, beta_hat_12, X_i)
                if temp2[i]:
                    A_2 = -A_2
    
        # if disagrees, we drop the point; else, we update estimators
        if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
            data.dropped[i] = False
            n_total = n_total + 1    
            #estimate hat parameters
            if n_total >= 2*d:           
                beta_hat_minus11, beta_hat_11, beta_hat_minus12, beta_hat_12 = compute_beta_linearQ(data)        
        else:
            data.dropped[i] = True 
        
        if n_total >= n:
            break
            
    # if we don't get enough samples, we omit this instance
    if n_total<n:
        return
            
    mean_reward = compute_mean_reward(data)
    return mean_reward

In [ ]:
eps_list = [0, 0.1, 0.2]
eps_greedy = {}
for i in range(n_reps):
    
    data = upweighted_data(tau, random_state = i)
                    
    for eps in eps_list:
        cur = epsilon_greedy(eps, d, n, random_state = i)
        if eps in eps_greedy:
            eps_greedy[eps].append(cur)
        else:
            eps_greedy[eps] = [cur] 

In [42]:
for key in eps_greedy:
    print(key, np.mean(eps_greedy[key]))

0 -7.81048
0.1 -7.723740000000001
0.2 -7.83481


## LSVI-UCB (Normalized)

In [43]:
def compute_w(Lambda, temp):
    return np.linalg.inv(Lambda) @ temp

In [44]:
def compute_Q(w, phi, beta, Lambda):
    temp = w.T @ phi + beta * (phi.T @ np.linalg.inv(Lambda) @ phi)**(1/2)
    return temp[0][0]

In [45]:
def compute_temp1(a, data, w2_1, w2_minus1, beta, Lambda2_1, Lambda2_minus1):
    subdata = data[(data.dropped == False)  & (data.L2_treatment == a)]
    temp1 = np.zeros([4,1])
    for i in subdata.index:
        phi1 = np.array([1, 
                         subdata.at[i,"L2_Int_QCTOT"], 
                         subdata.at[i,"L1_slope"], 
                         subdata.at[i,"L2_preference"]]
                       ).reshape(4,1)
        phi2 = np.array([1, 
                         subdata.at[i,"L3_Int_QCTOT"], 
                         subdata.at[i,"L2_slope"], 
                         subdata.at[i,"L3_preference"]]
                       ).reshape(4,1)
        if a == 1:
            max_Q = max(compute_Q(w2_1, phi2, beta, Lambda2_1), 
                    compute_Q(w2_minus1, phi2, beta, Lambda2_minus1)
                   )
        else:
            max_Q = compute_Q(w2_minus1, phi2, beta, Lambda2_minus1)
        temp1 += phi1 * (subdata.at[i,"L2_End_QCTOT"] + max_Q)
    return temp1

In [46]:
def optimal_arm(Q_minus1, Q_1):
    if Q_minus1 > Q_1:
        return -1
    return 1

In [47]:
def normalized_data(tau, random_state):
    data = upweighted_data(tau, random_state)
    data.L2_Int_QCTOT = data.L2_Int_QCTOT/26
    data.L2_End_QCTOT = data.L2_End_QCTOT/26
    data.L3_Int_QCTOT = data.L3_Int_QCTOT/26
    data.L3_End_QCTOT = data.L3_End_QCTOT/26
    data.L1_slope = data.L1_slope/10
    data.L2_slope = data.L2_slope/10
    return data

In [48]:
def lsvi_ucb_normalized(lamb, beta, d, n):
    
    # we include an intercept here, so d should be set to d+1
    
    data['dropped'] = None
    n_total = 0 #total number of points
    
    Lambda2_1 = lamb * np.identity(d)
    Lambda2_minus1 = lamb * np.identity(d)
    Lambda1_1 = lamb * np.identity(d)
    Lambda1_minus1 = lamb * np.identity(d)
    temp2_1 = np.zeros([d,1])
    temp2_minus1 = np.zeros([d,1])

    
    for i in data.index:
    
        # choose the optimal arm at stage 2
        phi2 = np.array([1, 
                        data.at[i,"L3_Int_QCTOT"], 
                        data.at[i,"L2_slope"], 
                        data.at[i,"L3_preference"]]).reshape(d,1)
        w2_1 = compute_w(Lambda2_1, temp2_1)
        w2_minus1 = compute_w(Lambda2_minus1, temp2_minus1)
        if - data.L2_End_QCTOT[i] < 6/26:
            A_2 = 0
        elif data.L2_treatment[i] == 1:
            Q2_1 = compute_Q(w2_1, phi2, beta, Lambda2_1)
            Q2_minus1 = compute_Q(w2_minus1, phi2, beta, Lambda2_minus1)
            A_2 = optimal_arm(Q2_minus1, Q2_1)
        else:
            A_2 = -1
        
            
        # if agrees
        if A_2 == data.L3_treatment[i]:
            # choose the optimal arm at stage 1
            phi1 = np.array([1, 
                             data.at[i,"L2_Int_QCTOT"], 
                             data.at[i,"L1_slope"], 
                             data.at[i,"L2_preference"]]).reshape(d,1)
            temp1_1 = compute_temp1(1, data, w2_1, w2_minus1, beta, Lambda2_1, Lambda2_minus1)
            temp1_minus1 = compute_temp1(-1, data, w2_1, w2_minus1, beta, Lambda2_1, Lambda2_minus1)
            w1_1 = compute_w(Lambda1_1, temp1_1)
            w1_minus1 = compute_w(Lambda1_minus1, temp1_minus1)
            Q1_1 = compute_Q(w1_1, phi1, beta, Lambda1_1)
            Q1_minus1 = compute_Q(w1_minus1, phi1, beta, Lambda1_minus1)
            A_1 = optimal_arm(Q1_minus1, Q1_1)
            
            # if agrees, update parameters
            if A_1 == data.L2_treatment[i]:
                data.dropped[i] = False
                n_total = n_total + 1
                
                Lambda2_1 += (phi2 @ phi2.T) * (A_2 == 1)
                Lambda2_minus1 += (phi2 @ phi2.T) * (A_2 == -1)
                Lambda1_1 += (phi1 @ phi1.T) * (A_1 == 1)
                Lambda1_minus1 += (phi1 @ phi1.T) * (A_1 == -1)
                temp2_1 += (phi2 * data.at[i,"L3_End_QCTOT"]) * (A_2 == 1)
                temp2_minus1 += (phi2 * data.at[i,"L3_End_QCTOT"]) * (A_2 == -1)
            else:
                data.dropped[i] = True  
        else:
            data.dropped[i] = True 
        
        if n_total >= n:
            break
            
    # if we don't get enough samples, we omit this instance
    if n_total<n:
        return
                
    mean_reward = compute_mean_reward(data)
    return mean_reward

In [ ]:
beta = 4 * 2 * (math.log(2* 4 * n ))**(1/2)
beta_list = [beta]

lsvi = {}
for i in range(n_reps):
    
    data = normalized_data(tau, random_state = i)
                    
    for beta in beta_list:
        cur = lsvi_ucb_normalized(1, beta, 4, n)
        if beta in lsvi:
            lsvi[beta].append(cur)
        else:
            lsvi[beta] = [cur] 

In [50]:
for key in lsvi:
    print(key, np.mean(lsvi[key]))

26.04197809149967 -8.07054


## Offline

In [51]:
def offline(n):
    data['dropped'] = None
    n_total = 0 #total number of points

    #the first period beta's are 0
    for i in data.index:    
        
        if -0.73 - 0.01* data.L2_Int_QCTOT[i] + 0.01*data.L1_slope[i] - 0.67*data.L2_preference[i]>0:
            A_1 = 1
        else:
            A_1 = -1
        
        if - data.L2_End_QCTOT[i] < 6:
            A_2 = 0
        elif A_1 == -1:
            A_2 = -1
        elif -0.18 + 0.01* data.L3_Int_QCTOT[i] - 0.25* data.L2_slope[i] >0 :
            A_2 = 1
        else:
            A_2 = -1
       
        # if disagrees, we drop the point; else, we update estimators
        if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
            data.dropped[i] = False
            n_total = n_total + 1      
        else:
            data.dropped[i] = True 
        
        if n_total >= n:
            break
            
    # if we don't get enough samples, we omit this instance
    if n_total<n:
        return
            
    mean_reward = compute_mean_reward(data)
    return mean_reward

In [ ]:
off = []
for i in range(n_reps):
    
    data = upweighted_data(tau, random_state = i)
    
    cur = offline(n)
    off.append(cur)

In [53]:
np.mean(off)

-7.688250000000001